In [ ]:
#!pip install snorkel
#!pip install dask distributed --upgrade
#!python -m spacy download en_core_web_md

In [1]:
from tqdm import tqdm
import pandas as pd 
import os
from snorkel.labeling import PandasLFApplier,LFAnalysis,LabelingFunction
from snorkel.labeling.model.label_model import LabelModel
from snorkel.labeling.apply.dask import PandasParallelLFApplier
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import time
import datetime
from datetime import datetime, date, time
from nltk import ngrams

In [15]:
#path information
task='task1'# or 'task2' # specify task
root_path='./'
#root_path='/repo1/code/autoreview/'
data_path=root_path+'data/'+task+'/'#path to save retrieved articles abstract
keyword_path=data_path+'keywords/'#keyword list
save_path=root_path+'results/'+task+'/'
literature_file_name='metadata_hypercoagulable.tsv'
pseudo_label_file_name='pseudo_label.pkl'

In [3]:
def build_sentence_df():
    """
    Read articles files (data_path+article_name) and save it as `sentences` a sentence-wise pd.dataframe
    Refined from `build_raw_data`
    
    output:
    """
    articles=pd.read_csv(data_path+literature_file_name, sep='\t').drop(columns='Unnamed: 0')
    
    sentences=articles['abstract'].apply(sent_tokenize)\
                    .apply(pd.Series)\
                    .merge(articles, left_index = True, right_index=True)\
                    .drop('abstract', axis=1)\
                    .melt(id_vars=['ncord_uid'], value_name='sent')\
                    .drop('variable', axis=1).dropna()
    
    return articles[['ncord_uid', 'abstract']], sentences


In [4]:
def load_keyword(keyword_file_name):
    """
    Read keyword list
    
    input:
        keyword_type_name: str, file name, e.g., `keylist.txt`
    output:
        list of string, e.g., ['keyword1', 'keyword2', 'keyword3']
        
    """
    with open(keyword_path+keyword_file_name, "r") as f:
        keylist=f.read().split(',')
    return keylist

In [22]:
def loop_labeling(col, keywordslist,viruslist,triallist,mustlist,maxngramnu):
    
    # keywordslist is the concept we want
    # viruslist is what type of virus we want to loop up
    # triallist is concept of trial we want to loop up
    # mustlist is concept of  we want the high ranking sentence should have
    # maxngramnu is the maxinum number of word window
    
    # lookup keywords in n-words window (for controling the window, we set up the max number of word window)
    def keyword_lookup(x,keywords,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x[col].lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Abstain

    def make_keyword_lf(keywords,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_{name}",
            f=keyword_lookup,
            resources=dict(keywords=keywords,maxngram=maxngram,label=label),)
    
    # lookup both group keywords in n-words window (for controling the window, we set up the max number of word window)
    
    def keyword_lookup_b(x,keywords1,keywords2,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x[col].lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords1) and any(word1 in wordlist  for word1 in keywords2):
                    threshold=threshold+1
        if threshold>0:
            return label
        return Abstain

    def make_keyword_b_lf(keywords1,keywords2,maxngram,name,label=None):
        return LabelingFunction(
            name=f"keyword_b_{name}",
            f=keyword_lookup_b,
            resources=dict(keywords1=keywords1,keywords2=keywords2,maxngram=maxngram,label=label),)
   
    # lookup a group keywords presenting and not presenting in another group of keywords
    
    def not_lookup_b(x,keywords1,keywords2,maxngram,label):
        threshold=0
        for i in range(maxngram):
            gramab=ngrams(x[col].lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords2):
                    threshold=threshold+1
        threshold1=0
        for i in range(maxngram):
            gramab=ngrams(x[col].lower().split(), i+1)
            for word in gramab:
                wordlist=list(word)
                if any(word1 in wordlist  for word1 in keywords1):
                    threshold1=threshold1+1
        if threshold<1 and threshold1>0:
            return label
        return Abstain

    
    def make_not_b_lf(keywords1,keywords2,name,maxngram,label=None):
        return LabelingFunction(
            name=f"not_b_{name}",
            f=not_lookup_b,
            resources=dict(keywords1=keywords1,keywords2=keywords2,maxngram=maxngram,label=label),)   
    
    
      # lookup up number in sentence 
    
    
    def number_lookup(x,keywords,label):
        threshold=0
        sentlist=x[col].split()
        trialindex=[i for i, e in enumerate(sentlist) if e in keywords]
        nuindex=[i for i, e in enumerate(sentlist) if str(e).isdigit()]
        for i in trialindex:
            for j in nuindex:
                if i<j:
                    threshold+=1
        if threshold>0:
            return label
        return Abstain

    def make_number_lf(keywords,label=None):
        return LabelingFunction(
            name=f"number",
            f=number_lookup,
            resources=dict(keywords=keywords,label=label),)

   
    
    Abstain = -1
    
    keywordfu=make_keyword_lf(keywords=keywordslist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfu=make_keyword_lf(keywords=viruslist,maxngram=maxngramnu,name='virus',label=1)
    keywordfub=make_keyword_b_lf(keywords1=keywordslist,keywords2=mustlist,maxngram=maxngramnu,name='keyword',label=1)    
    virusfub=make_keyword_b_lf(keywords1=viruslist,keywords2=mustlist,maxngram=maxngramnu,name='virus',label=1)
    trialfu=make_keyword_lf(keywords=triallist,name='trial',maxngram=maxngramnu,label=1)
    notfu1=make_not_b_lf(keywords1=mustlist,keywords2=keywordslist,maxngram=maxngramnu,name='notkeyword',label=0)
    notfu2=make_not_b_lf(keywords1=keywordslist,keywords2=triallist,maxngram=maxngramnu,name='nottrial',label=0)
    numberfu=make_number_lf(keywords=triallist,label=1)
    
    allweaklabf=[]

    allweaklabf.append(keywordfu)
    allweaklabf.append(virusfu)
    allweaklabf.append(keywordfub)
    allweaklabf.append(virusfub)
    allweaklabf.append(trialfu)
    allweaklabf.append(notfu1)
    allweaklabf.append(notfu2)
    allweaklabf.append(numberfu)

    
    return allweaklabf

In [6]:
def predict_prob(df, lfs, cardinality=2):
    """
    Predict probability (label) by applying label functions lfs
    Refined from `snorkel_process` 
    
    input:
        lfs: list of snorkel.LabelingFunction
    output:
        sentences_labeled: pd.Dataframe,  [pid, sid, sent, label, prob ]
    """
    
    applier=PandasLFApplier(lfs=lfs)
    applied=applier.apply(df=df)
    print(LFAnalysis(L=applied, lfs=lfs).lf_summary())
    
    label_model = LabelModel(cardinality=cardinality,verbose=True)
    label_model.fit(applied)
    df['label']=label_model.predict(applied)
    df['prob']=label_model.predict_proba(applied)[:,1]
    
    return df

In [7]:
### make pd.Dataframe for sentences
abstracts, sentences = build_sentence_df()

Load keywords

In [8]:
keywordslist=load_keyword('keywords.txt')
viruslist=load_keyword('viruslist.txt')
triallist=load_keyword('triallist.txt')
mustlist=['hypercoagulable']

Generate labeling functions

In [13]:
allweaklabf=loop_labeling('abstract',keywordslist,viruslist,triallist,mustlist,3)

Apply labeling functions 

In [14]:
abstracts_prob=predict_prob(abstracts,allweaklabf)


 12%|█▏        | 907/7862 [00:17<02:14, 51.73it/s]


 23%|██▎       | 1792/7862 [00:37<01:58, 51.23it/s]


 35%|███▍      | 2733/7862 [00:55<01:32, 55.29it/s]


 47%|████▋     | 3663/7862 [01:13<01:13, 56.85it/s]


 58%|█████▊    | 4547/7862 [01:32<04:35, 12.02it/s]


 69%|██████▉   | 5424/7862 [01:52<00:48, 50.69it/s]


 81%|████████  | 6329/7862 [02:11<00:29, 52.02it/s]


 92%|█████████▏| 7225/7862 [02:30<00:13, 48.35it/s]


100%|██████████| 7862/7862 [02:42<00:00, 48.30it/s]


                   j Polarity  Coverage  Overlaps  Conflicts
keyword_keyword    0      [1]  0.138387  0.138387   0.031417
keyword_virus      1      [1]  0.082040  0.066268   0.009540
keyword_b_keyword  2       []  0.000000  0.000000   0.000000
keyword_b_virus    3      [1]  0.000636  0.000636   0.000636
keyword_trial      4      [1]  0.717756  0.462605   0.001018
not_b_notkeyword   5      [0]  0.001145  0.001145   0.001145
not_b_nottrial     6      [0]  0.031417  0.031417   0.031417
number             7      [1]  0.401933  0.401933   0.000254


Save it

In [16]:
abstracts_prob=abstracts_prob.sort_values(by='prob',ascending=False)
abstracts_prob.to_pickle(save_path+pseudo_label_file_name)